## Exercise 1

Consider the quadratic programme

$$
\begin{array}{ll}
\min & x_1^2 + 2 x_2^2 - x_1 x_2 - x_1\\
s.t. & x_1 + 2x_2 \le u_1,\\
& x_1 - 4x_2 \le u_2,\\
& x_1 + x_2 \ge -5,
\end{array} \tag{1}
$$

with variables $x_1$, $x_2$ and parameters $u_1$ and $u_2$.

1.   Solve this QP, for parameter values $u_1 = -2$ and $u_2 = -3$. Print the optimal primal variables $x_1^*$ and $x_2^*$, the optimal dual variables $\lambda_1^*$, $\lambda_2^*$ and $\lambda_3^*$, and the optimal objective value $p^*$. 
2.   Verify that the KKT conditions hold for the optimal primal and dual variables you found (within reasonable numerical accuracy).
3.   Consider the perturbed problem with 
$$
u_1 = -2 + \delta_1,\qquad u_2=-3+\delta_2,
$$
where $\delta_1$ and $\delta_2$ take both values in the set $\{-0.1,0,0.1\}$ (there are a total of nine such combinations, including the original problem). For each combination of $\delta_1$ and $\delta_2$, make a prediction $p^*_{pred}$ of the optimal value of the perturbed QP, and compare it to $p^*_{exact}$ which is the exact optimal value of the perturbed QP (obtained by solving the perturbed QP). Complete the table:
$$
\begin{array}{|c|c|c|c|c|c|}\hline
& \delta_1 & \delta_2 & p^*_{pred} & p^*_{exact} & p^*_{exact}-p^*_{pred}\\\hline
1 & 0 & 0 & & & \\\hline
2 & 0 & -0.1 & & & \\\hline
3 & 0 & 0.1 & & & \\\hline
4 & -0.1 & 0 & & & \\\hline
5 & -0.1 & -0.1 & & & \\\hline
6 & -0.1 & 0.1 & & & \\\hline
7 & 0.1 & 0 & & & \\\hline
8 & 0.1 & -0.1 & & & \\\hline
9 & 0.1 & 0.1 & & & \\\hline
\end{array}
$$
Check that the inequality $p^*_{pred} \le p^*_{exact}$. For which perturbations (other than number $1$) is $p^*_{exact}-p^*_{pred}$ the smallest? *Tip: round to 5 decimals.*





In [ ]:
import cvxpy as cp
import numpy as np

CVXPY doesnt recognise x1*x2 as a convex function, so you need to write the program in quadratic form

In [ ]:
#Part 1
x = cp.Variable(2)
P = [[1, -1/2], [ -1/2, 2]]

G = [[1,2], [1,-4], [1,1]]
q = [-1,0]
h = [-2,-3,5]



u1 = -2
u2 = -3

constraints = [x1 + 2*x2 + u1 <= 0,
               x1 - 4*x2 + u2 <=0,
               x1 + x2 + 5 >= 0 ]

obj = cp.Minimize(x1 + 2*(x2**2) - x1*x2 - x1)

prob = cp.Problem(obj, constraints)

prob.solve()

/usr/local/lib/python3.8/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


DCPError: ignored

### Solution Exercise 1

We solve the QP and find the primal and dual optimal variables, and the optimal value. The easiest way of solving this problem is by writing the QP in matrix form, i.e. finding matrices $P$, $G$, $q$ and $h$ such that

$$
\begin{array}{ll}
\min & \frac{1}{2}x^\top P x + q^\top x \\
s.t. & G x \preccurlyeq h 
\end{array}
$$

To simplify the code, in the following we include $\frac{1}{2}$ within $P$.

Note that if we do not write the problem in matrix form we will have problems in the objective function when we try to evaluate $x_1 x_2$. In fact, CVXPY does not recognise this term as an operation following DCP rules (the same holds for $x_1^2$, but this specific issue can be avoided with `cp.square`).

In [ ]:
# Import libraries
import cvxpy as cp
import numpy as np

In [ ]:
# Declare variables and matrices of the QP
x = cp.Variable(2)
P = np.array([[1, -1/2], [-1/2, 2]])
G = np.array([[1, 2], [1, -4], [-1, -1]])
q = np.array([-1, 0])
h = np.array([-2, -3, 5])

In [ ]:
# Define constrains, problem and solve it
constraints = [G @ x <= h]
prob = cp.Problem(cp.Minimize(cp.quad_form(x, P)+ q.T @ x),constraints)
prob.solve()

#Find optimal value and dual variables
OptVal = prob.value
DualVar = constraints[0].dual_value

# Print result.
print("\nThe optimal value is", np.round(OptVal,2))
print("The optimal x is",np.round(x.value,2))
print("Dual variables", np.round(DualVar,2))


The optimal value is 8.22
The optimal x is [-2.33  0.17]
Dual variables [3.39 2.44 0.  ]


We now verify that the KKT conditions hold for the optimal primal and dual variables found.

In [ ]:
x1=x.value[0]
x2=x.value[1]
l1=DualVar[0]
l2=DualVar[1]
l3=DualVar[2] 

print('KKT conditions:')
print("Constraint nonpositivity: ", G @ x.value - h <=0)
print("Nonnegativity of dual variables: ", DualVar >=0)
print("Complementary: ", DualVar * (G @ x.value-h) == 0)
print("Gradient: ", 2*x.value.T@P+q.T+DualVar@G==0)
#print("Gradient: ", np.array([[2*x1-x2-1],[4*x2 - x1]])+np.array([[l1+l2-l3],[2*l1-4*l2-l3]])==0)

KKT conditions:
Constraint nonpositivity:  [ True  True  True]
Nonnegativity of dual variables:  [ True  True  True]
Complementary:  [ True  True  True]
Gradient:  [ True  True]


We now solve the perturbed problem

In [ ]:
# Generate the solutions and prediction for each of the 9 perturbed problems.
delta = np.array([0,-0.1,0.1])
N=3
for i in range(N):
  for j in range(N):
    h = np.array([-2+delta[i], -3+delta[j], 5])
    constraints = [G @ x <= h]
    prediction = OptVal - DualVar @ np.array([delta[i], delta[j], 0])
    prob = cp.Problem(cp.Minimize(cp.quad_form(x, P)+ q.T @ x),constraints)
    prob.solve()
    print("Line", i*3+j+1 ,"- Delta1:",delta[i], "Delta2:", delta[j]  , "Prediction:", 
          np.round(prediction,5),  "Exact value:",np.round(prob.value,5), "Prediction gap:", np.round(prob.value - prediction,5))

Line 1 - Delta1: 0.0 Delta2: 0.0 Prediction: 8.22222 Exact value: 8.22222 Prediction gap: 0.0
Line 2 - Delta1: 0.0 Delta2: -0.1 Prediction: 8.46667 Exact value: 8.46889 Prediction gap: 0.00222
Line 3 - Delta1: 0.0 Delta2: 0.1 Prediction: 7.97778 Exact value: 7.98 Prediction gap: 0.00222
Line 4 - Delta1: -0.1 Delta2: 0.0 Prediction: 8.56111 Exact value: 8.565 Prediction gap: 0.00389
Line 5 - Delta1: -0.1 Delta2: -0.1 Prediction: 8.80556 Exact value: 8.81556 Prediction gap: 0.01
Line 6 - Delta1: -0.1 Delta2: 0.1 Prediction: 8.31667 Exact value: 8.31889 Prediction gap: 0.00222
Line 7 - Delta1: 0.1 Delta2: 0.0 Prediction: 7.88333 Exact value: 7.88722 Prediction gap: 0.00389
Line 8 - Delta1: 0.1 Delta2: -0.1 Prediction: 8.12778 Exact value: 8.13 Prediction gap: 0.00222
Line 9 - Delta1: 0.1 Delta2: 0.1 Prediction: 7.63889 Exact value: 7.64889 Prediction gap: 0.01


All prediction gaps are nonnegative, so the inequality $p^*_{pred} \le p^*_{exact}$ is satisfied. The best predictions are achieved for perturbations number $2$, $3$, $6$ and $8$.

# Exercise 2

This exercise is mostly done with pen and paper.

Consider the optimisation problem

$$
\begin{array}{ll}
\min & x^2+1\\
s.t. & (x-2)(x-4) \le 0,
\end{array}
$$

with variable $x\in\mathbb{R}$.
1.    Give the feasible set, the optimal value, and the optimal solution. Plot the objective $x^2+1$ versus $x$. On the same plot, show the feasible set, optimal point and value.
2.    Plot the Lagrangian $L(x, \lambda)$ versus $x$ for a few positive values of $\lambda$. Verify the lower bound property ($p^* \ge \inf_x L(x, \lambda)$ for $\lambda\ge 0$). Derive and sketch the Lagrange dual function $g$.
3.    State the dual problem, and verify that it is a concave maximization problem. Find the dual optimal value and dual optimal solution $\lambda^*$. Does strong duality hold?
4.    Let $p^*(u)$ denote the optimal value of the perturbed problem
$$
\begin{array}{ll}
\min & x^2+1\\
s.t. & (x-2)(x-4) \le u
\end{array}
$$
as a function of the parameter $u$. Plot $p^*(u)$. Verify that $\frac{dp^*}{du}(0) = −\lambda^*$.

## Exercise 3

Find the dual function of the LP
$$
\begin{array}{ll}
\min & c^\top x\\
s.t. & G x \preceq h\\
& Ax =b.
\end{array}
$$
Give the dual problem, and make the implicit equality constraints explicit.